In [14]:
# Run of the model for all the samples generated by the morris sampling

from pandas.core.api import value_counts

# Importazione dei dati

import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
pd.read_excel("Parameters.xlsx", sheet_name="Foglio1")

ode = pd.read_excel("Parameters.xlsx", sheet_name="Foglio1", usecols='A:E')



for indice, riga in ode.iterrows():
    nome_riga = riga["Symbol"]
    valore = float(riga["Converted value"])
    globals()[nome_riga] = valore

In [15]:
#Import of the sample
percorso_file_excel = 'location of the sample'


df = pd.read_excel(percorso_file_excel)

inizio = 0.0
fine = 21600.0 #6 ore
passo = 1 # 1s
num = int((fine - inizio) / passo) # numero di punti
time = np.linspace(inizio,fine,num) # asse temporale
ct_totale = np.zeros((df.shape[0],num))
SF_totale = np.zeros((df.shape[0],num))
r_totale = np.zeros((df.shape[0],num))
parametri_curve = np.zeros((df.shape[0],7))

In [16]:
def sigm(t,X,Y,Z,D):
  s = X - ((Y)/(1 + np.exp(-Z * (t - D))))
  return s

from scipy.optimize import curve_fit
def iperbole(x, a, b, cc):
    return a / (x + b) + cc

In [17]:
#Run of the model
for xxx in range(df.shape[0]):
   
    
    riga_selezionata = df.loc[xxx]
    for colonna in df.columns:
        globals()[colonna] = riga_selezionata[colonna]
    


    #Definisco le funzioni
    K1_tpz = 0.0
    K2_tpz = 0.0
    K3_tpz = P_tpz * D_tpz / (P_tpz * L + D_tpz) * (SV)
    K1_ox = 0
    K2_ox = 0
    K3_ox = (P_ox * D_ox)/(D_ox + P_ox * L) *SV


    #Definisco la variabilità di cv
    inizio = 0.0
    fine = 21600.0 #6 ore
    passo = 1 # 1s
    num = int((fine - inizio) / passo) # numero di punti
    time = np.linspace(inizio,fine,num) # asse temporale
    # Decrescita esponenziale del farmaco
    def decay(t):
        c_v_decay = c_v0_tpz * np.exp(-t * (1/tau) )
        return c_v_decay

    ca = (c_v0_tpz - 0)/3600 # coefficiente angolare salita concentrazione per circa un'ora fino al valore di regime c_v da letteratura

    c_vv_rise = np.linspace(0, 3600, int(3600 / passo)) * ca
    c_vv_const = np.ones(int(3600 / passo), dtype=float) * c_v0_tpz # fase costante di c_v durante la somministrazione, considerata per la metà del tempo rimanente
    decay_time = np.linspace(0, fine - 7200, int((fine - 7200) / passo))  # studio del decadimento per il tempo restante
    c_vv_dec = decay(decay_time)
    c_vv = np.concatenate([c_vv_rise, c_vv_const, c_vv_dec])



    #Resolution of the problem
    def f_ox(c,SF):
        dcdt = -c * (K1_ox - K2_ox + K3_ox+ SF*(V_max_ox/(K_m_ox + c))) + (K2_ox * c_v0_ox + K3_ox * c_v0_ox)
        return dcdt
    

    def f(c, suma, c_vv, ox):
        dcdt = -c * (K1_tpz - K2_tpz + K3_tpz) - (K / (K + ox)) * (k_met * c + (V_max_tpz * c) / (K_m_tpz + c)) * np.exp(-alpha * suma) * phi_0 + K2_tpz * c_vv + K3_tpz * c_vv
        return dcdt
    
    c = [0.0]
    SF = [] 
    aree = []
    consumo_nl = [0.0]
    m_app_cvvar = []
    V_max_app_cvvar = []
    flag = 0
    consumo = []
    consumosf = []
    consumonoc = []
    consumonocnosf = []

    CM = [0.0]
    c_ox = [0.0042344]
    for i in time:
        suma = np.sum(aree)
        c_temp = float(c[-1])
        k1 = f(c_temp, suma,c_vv[flag],c_ox[-1])
        k2 = f((c[-1] + (k1 / 2)*passo).astype(np.float64), suma, c_vv[flag],c_ox[-1])
        k3 = f((c[-1] + (k2 / 2)*passo).astype(np.float64), suma, c_vv[flag],c_ox[-1])
        k4 = f((c[-1] + k3*passo).astype(np.float64), suma, c_vv[flag],c_ox[-1])
        c_temp = c[-1] + (passo / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
        SF_temp = np.exp(-alpha * suma)
        SF.append(SF_temp)
        c.append(c_temp)
        CM.append((K / (K + c_ox[-1])) * c_temp * (k_met * c_temp + V_max_tpz * c_temp / (K_m_tpz + c_temp)))
        consumonocnosf.append((K / (K + c_ox[-1])) * (k_met  + V_max_tpz  / (K_m_tpz + c_temp)) * phi_0)
        area = ((float(CM[-1]) + float(CM[-2]))*float(passo))/2
        aree.append(area)
        c_temp_ox = float(c_ox[-1])
        k1_ox = f_ox(c_temp_ox,float(SF_temp))
        k2_ox = f_ox((c_ox[-1] + (k1_ox / 2)*passo),float(SF_temp))
        k3_ox = f_ox((c_ox[-1] + (k2_ox / 2)*passo),float(SF_temp))
        k4_ox = f_ox((c_ox[-1] + k3_ox*passo),float(SF_temp))
        c_temp_ox = c_ox[-1] + (passo / 6) * (k1_ox + 2 * k2_ox + 2 * k3_ox + k4_ox)
        c_ox.append(c_temp_ox)
        flag = flag + 1
    
    X = max(SF)
    Y = X - min(SF)
    Z = 0.0005
    D = len(time)/2
    params, covariance = curve_fit(sigm, time, np.array(SF).flatten(), p0=[X, Y, Z, D])
    XX, YY, ZZ, DD = params

    params, covariance = curve_fit(iperbole, time, np.array(consumonocnosf).flatten())
    a, b, cc = params

    parametri_curve[xxx] = XX, YY, ZZ, DD, a, b, cc 
    ct_totale[xxx] = np.array(c[0:-1]).flatten()
    SF_totale[xxx] = np.array(SF).flatten()
    r_totale[xxx] = np.array(consumonocnosf).flatten()
    clear_output(wait=False)
    print(f"{xxx}/{df.shape[0] - 1}")


559/559


In [22]:
#Save of the results
np.save('ct_totale_curveper3D.npy', ct_totale)
np.save('SF_totale_curveper3D.npy', SF_totale)
np.save('r_totale_curveper3D.npy', r_totale)
np.save('parametri_curve_per3D.npy', parametri_curve)
from scipy.io import savemat



''' # Salva l'array come matrice MATLAB
savemat('Traiettorie_ct.mat', {'Traiettorie_ct': ct_totale})
savemat('Traiettorie_SF.mat', {'Traiettorie_SF': SF_totale})
savemat('Traiettorie_r.mat', {'Traiettorie_r': r_totale})
savemat('parametri_curve.mat', {'parametri_curve': parametri_curve})'''

parametri_curve.shape

(560, 7)

In [32]:
#Dataframe fror the parameters of the curves
stringhe_variabili = df.columns
valori_stringa = ['XX_sigmoide','YY_sigmoide','ZZ_sigmoide','DD_sigmoide','a_iperbole','b_iperbole','cc_iperbole']
parametri_pd = pd.Index(valori_stringa)

campi_df = stringhe_variabili.append(parametri_pd)
campi_df
dati_numpy_trasposti = parametri_curve.T
dati_parametri = pd.DataFrame(data=parametri_curve, columns=parametri_pd)
dati_parametri
df_concatenato = pd.concat([df, dati_parametri],axis=1, ignore_index=False)
df_concatenato


Index(['c_v0_tpz', 'k_met', 'K', 'alpha', 'c_v0_ox', 'V_max_ox', 'P_ox',
       'XX_sigmoide', 'YY_sigmoide', 'ZZ_sigmoide', 'DD_sigmoide',
       'a_iperbole', 'b_iperbole', 'cc_iperbole'],
      dtype='object')

In [48]:
#Save of the parameters
df_trasposto_t = df_concatenato.T
df_trasposto_t.to_csv('sample_e_coefficienti_csv.csv', index=True)
df_trasposto_t.to_excel('sample_e_coefficienti_excel.xlsx', index=True)
df_trasposto_t.to_csv('sample_e_coefficienti_noindici_csv.csv', index=False)
nomi_colonne_df = pd.DataFrame({'Nomi Colonne': df_concatenato.columns})


percorso_file_excel = 'nomi_colonne.xlsx'


nomi_colonne_df.to_excel(percorso_file_excel, index=False)

In [19]:

ct_media = np.zeros((ct_totale.shape[0],))
SF_media = np.zeros((SF_totale.shape[0],))
r_media = np.zeros((r_totale.shape[0],))

In [20]:
#Averaging the quantities
for i in range(ct_totale.shape[0]):
    ct_media[i] = np.mean(ct_totale[i,:])
    SF_media[i] = np.mean(SF_totale[i,:])
    r_media[i] = np.mean(r_totale[i,:])
    clear_output(wait=False)
    print(f"{i}/{ct_totale.shape[0] - 1}")



np.save('safe_R1.1\Simulazione_1\ct_media_curveper3D.npy',ct_media)
np.save('safe_R1.1\Simulazione_1\SF_media_curveper3D.npy',SF_media)
np.save('safe_R1.1\Simulazione_1\\r_media_curveper3D.npy',r_media)

'''
savemat('safe_R1.1\Simulazione_1\ct_media.mat', {'ct_media': ct_media})
savemat('safe_R1.1\Simulazione_1\SF_media.mat', {'SF_media': SF_media})
savemat('safe_R1.1\Simulazione_1\\r_media.mat', {'r_media': r_media}) '''
    

559/559


"\nsavemat('safe_R1.1\\Simulazione_1\\ct_media.mat', {'ct_media': ct_media})\nsavemat('safe_R1.1\\Simulazione_1\\SF_media.mat', {'SF_media': SF_media})\nsavemat('safe_R1.1\\Simulazione_1\\r_media.mat', {'r_media': r_media}) "